此次專案 dataset is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. 現在我們手上有 3份資料：
- Enhanced Twitter Archive
- Additional Data via the Twitter API
- Image Predictions File

### Gather

依照課程提供，我有兩份資料可以直接用 `pandas` 打開
- twitter-archive-enhanced.csv
- image-predictions.tsv

另外還有一份是透過 twitter api 取得資訊，取用的程式我寫在 `tweet.py`，得到一份資料存成 txt 檔，再用 `pandas` 進行讀取。
- tweet_json.txt

這一步我很快的就通過了，沒有碰到太大的困難。

### Access

這一個項目其實讓我卡關很久，一開始單純用 `pandas` method `head` or `info` 瀏覽資料，但是資料實在太多，無法在 jupyter notebook 有效瀏覽，後來我有用 Excel 開啟檔案，然而大量的資料讓我在一開始感到很無力，不知所措。

後來，我讓自己在 Excel 只挑選 twitter-archive-enhanced.csv 其中幾行資料來觀察，希望從裡面找到一些問題，也確實我發現了字面上的問題，我將其紀錄下來像是：
- doggo、floofer、pupper、puppo 內容是形容詞，可以改成 0, 1，此外還隱藏一個欄位，可以加總這 4 個欄位得知多個形容詞是否有不同結果
- retweeted_status_id、retweeted_status_user_id、retweeted_status_timestamp，推測是轉發推文id，可以等 archive table 和 tweet json 資料合併處理後再來檢查。

接著用 Excel 要開啟 tweet_json.txt 時，反而碰到另一個問題，因為 Excel 無法順利轉譯 json 格式，我又只好回頭使用 code 來作業，一打開又像是爆炸的資料，本來想使用整理 twitter-archive-enhanced.csv 觀察的概念來作業，但是並不是很順利。但還是嘗試列出一些項目：
- in_reply_to_screen_name、in_reply_to_status_id、in_reply_to_status_id_str、in_reply_to_user_id、in_reply_to_user_id_str 是同一個群組，當此訊息是回覆推文，會紀錄相關的訊息，那我們只考慮原始推文的資料，所以資料是回覆推文就要刪除。
- ...

第一次的紀錄詳情可以查看 [git commit "65d42e"](https://github.com/wrayz/hw-wrangle-and-analyze-data/blob/65d42edbb8a0a9a01d64c88ca9da3570eebcf50c/wrangle_report.ipynb)，內容使用中文撰寫可能不適合英文點閱，但我想從這個 commit 表達一件事情，在這之前的 Access 作業，我是**憑自己對內容的理解去猜測欄位的定義**，這個方式在做到 Clean 作業時，我就發現**這是錯誤的作法**。原因是當我質疑自己推測的依據是什麼？沒有依據就進行 clean，這樣的資料可以用來分析嗎？這些自問自答的結果都是**NO**！

為此，我上 Google 查找 twitter api document，很快就找到一份資料 [Tweet Objects](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html) 正是詳述 tweet_json.txt 的內容，這也代表我的 Access 作業要重來了，但我這次有了 twitter 依據就有了更多信心可以更好的去評估資料。

依照 document，我預計先重新 Accessing tweet_json.txt，再來是 twitter-archive-enhanced.csv，最後是 image-predictions.tsv，整理如下：

##### Tweet JSON table

**Quality issues**

打開資料第一件事，透過 `info` 查看有哪些 _missing data_ ，然後參考 the document of tweet object，我們列出這些欄位並描述如下：

- contributors、coordinates、geo 完全沒有資料，因此我們不考慮這些欄位。
- extended_entities 紀錄附件資訊，缺少這個欄位的資料，我們可以判定該推文並沒有圖片，因此不考慮這些沒有圖片的數據。
- in_reply_to_screen_name、in_reply_to_status_id、in_reply_to_status_id_str、in_reply_to_user_id、in_reply_to_user_id_str 是同一個群組，當此訊息是回覆推文，會紀錄相關的訊息，那我們只考慮原始推文的資料，所以資料是回覆推文就要刪除。
- possibly_sensitive、possibly_sensitive_appealable 紀錄敏感性資料，我認為這些欄位跟這次的主題沒有太大關係，不用特地保留。
- place 顯示推文地點資訊，但是只有 1筆，因此不保留此欄位。
- quoted_status、quoted_status_id、quoted_status_id_str、quoted_status_permalink 是同一群組，當訊息是引用推文，這些欄位就會紀錄相關訊息，一樣我們不要這些數據。
- retweeted_status: Retweets can be distinguished from typical Tweets by the existence of a retweeted_status attribute. 因此只要是轉發的數據，我們也不要。

接下來看其它欄位：

- display_text_range 表示文字範圍，不用這個欄位
- entities 原始推文的相關資訊，由於有 image prediction 找到相關圖片預測，因此也不用這個欄位
- favorited 全部都是 False，基本上也不用這個欄位
- id_str 型別錯誤，應為 string object
- is_quote_status: Indicates whether this is a Quoted Tweet. 因此只要是 True 我們不要
- lang 型別錯誤，應為 category datatype
- source 夾雜 html 標籤
- retweeted 都是 False，不用這個欄位
- truncated 都是 False，不用這個欄位
- user 都是同一個帳號 @WeRateDogs，這個欄位也可以移除

##### Twitter archive enhanced table

**Quaility issues**

_missing data_
- in_reply_to_status_id、in_reply_to_user_id 欄位資訊在 Tweet JSON 也有，我們可以合併完 table 後再來檢查數據是否為回覆訊息，是的話就要刪除
- retweeted_status_id、retweeted_status_user_id、retweeted_status_timestamp，推測是轉發推文id，可以等 archive table 和 tweet json 資料合併處理後再來檢查。
- expanded_urls，使用 Excel 查看內容發現有許多重複 url，目前不知道目的為何

_其他_
- Erroneous datatypes：timestamp 型別應改為 datetime
- source 和 Tweet JSON 的 source 相同，夾雜 html 標籤語言。

**Tidiness issues**
- doggo、floofer、pupper、puppo 內容是形容詞，可以改成 0, 1，此外還隱藏一個欄位，可以加總這 4 個欄位得知多個形容詞是否有不同結果
- rating_numerator, rating_denominator 還隱藏了一個欄位訊息 rating_numerator/rating_denominator 可以得到一個大致的比例

##### Image prediction table

**Quality issues**
-  有324筆數據判定不出是哪一種狗的品種

**Tidiness issues**
- 資料存在三種預測結果，裡面隱藏一個欄位即最終結果
- 這份資料可以跟 Twitter archive enhanced table 合併，將預測最好的結果（狗品種）合併過去。

### Clean

執行 clean 作業，原來是要依照 Accessing 列點逐一清理，但是我注意到
1. Accessing 的事項還要再歸納一次
2. 在 cleaning 時，必要的時候我還是需要重新做 Accessing 步驟，變成 cleaning-accessing-cleaning，這樣的目的都只是要確保我這麼做沒有太大的錯誤

以下我依照不同資料要 clean 的事項列點如下：

##### Tweet json table

- 移除不要的欄位，contributors、coordinates、geo、possibly_sensitive、possibly_sensitive_appealable、place、display_text_range、entities、favorited、retweeted、truncated、user
- 移除 extended_entities 缺失資料
- 刪除 in_reply_to_screen_name、in_reply_to_status_id、in_reply_to_status_id_str、in_reply_to_user_id、in_reply_to_user_id_str  有值的數據
- 刪除 is_quote_status 為 True 的數據
- 刪除 retweeted_status 有值的數據
- 檢查剩下資料，移除沒有資料的欄位
- 移除 is_quote_status、extended_entities 欄位
- 修正 id_str 型別為 str datatype
- 修正 lang 型別為 category datatype
- 修剪 source 內容不要 html 資訊

#### Twitter archive enhanced table
- 合併 Twitter archive enhanced table 和 Tweet json table。
- 合併後使用 info 檢查資訊，如果欄位沒有任何數據，可直接移除
- 計算 rating_numerator / rating_denominator 並放到新欄位 rating_ratio
- doggo、floofer、pupper、puppo 內容是形容詞，可以改成 0, 1，此外還隱藏一個欄位，可以加總這 4 個欄位得知多個形容詞是否有不同結果

#### Image prediction table
- 刪除查無品種的數據（一共 324筆）
- 依照預測結果優先順序找出狗品種並放到新開欄位 prediction 
- 與 Twitter archive enhanced table 合併

## Storing Data

最後我終於得到一份可以用來分析的乾淨資料！

資料我存成 twitter_archive_master.csv 檔，同時也存到 SqlLite，對我來說，這一步很輕鬆的就完成了。

In [1]:
import pandas as pd
df = pd.read_csv('twitter_archive_master.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1663 entries, 0 to 1662
Data columns (total 15 columns):
tweet_id          1663 non-null int64
text              1663 non-null object
expanded_urls     1663 non-null object
name              1663 non-null object
doggo             1663 non-null object
floofer           1663 non-null object
pupper            1663 non-null object
puppo             1663 non-null object
favorite_count    1663 non-null int64
lang              1663 non-null object
retweet_count     1663 non-null int64
source            1663 non-null object
created_at        1663 non-null object
rating_ratio      1663 non-null float64
breed             1663 non-null object
dtypes: float64(1), int64(3), object(11)
memory usage: 195.0+ KB


In [10]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'wrangle_report.ipynb'])

0